# ** Basic Cognitive Services Examples **

This set of examples is intended to illustrate the basics of calling the various cognitive serices apis via the SynapseML libraries.  These samples are intended to be functional with minimal external dependencies therefore they typically work on extremely small data sets and to not load external files except where absolutely necessary (ie. images).

The majority of these examples are taken from the Synapse ML examples: https://microsoft.github.io/SynapseML/docs/documentation/transformers/transformers_cognitive/ 
They have been modified slightly to account for pulling keys from key vault, and also to beautify the results and make them easier to read when executing in a jupyter notebook.  They are included here for the sake of completeness and also  

Several of the APIs illustrated in these examples will be illustrated in greater detail on other examples with larger datasets and leveraging external files.

## ***Prerequisites ***

1. A provisioned Synapse Workspace with a Spark 3.1 pool to run this notbook on.
1. A keyvault to store the cognitive services key necessary for executing these examples.
1. A provisioined cognitive services multi-endpont api instance.

## *** Setup ***

This first part is necessary to setup the cluster to leverage the SynapseML libraries which are loaded from Maven coordinates.

In [1]:
%%configure -f
{
  "name": "synapseml",
  "conf": {
      "spark.jars.packages": "com.microsoft.azure:synapseml_2.12:0.9.5-13-d1b51517-SNAPSHOT",
      "spark.jars.repositories": "https://mmlspark.azureedge.net/maven",
      "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.12,org.scalactic:scalactic_2.12,org.scalatest:scalatest_2.12",
      "spark.yarn.user.classpath.first": "true"
  }
}

StatementMeta(, 25, -1, Finished, Available)

To start we'll import a bunch of necessary modules and setup values for some resoruces like storage accounts, container, and keyvault that we'll use throughout the rest of the example.

In [2]:
from synapse.ml.cognitive import *
import pyspark
from pyspark.sql.functions import *  #needed for basic column manipulation like flattening json

#Input your key vault information.  The secret we need is for the cognitive services multi-service resource.
#set the name of your key vault
keyVaultName = "cgmsynapsevault"
#set the region where you cognitive service is deployed
serviceLocation = "eastus2"
cogServicesKey = mssparkutils.credentials.getSecret(keyVaultName, "cogsvcskey")



StatementMeta(synapsemlpool, 25, 1, Finished, Available)

## ** Computer Vision Service **

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/

*** OCR ***

The Optical character recognition extracts text from an image.

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/overview-ocr

In [3]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test3.png", ),
    ], ["url", ])

ocr = (OCR()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setImageUrlCol("url")
        .setDetectOrientation(True)
        .setOutputCol("ocr")
        .setErrorCol("error"))

results = ocr.transform(df).select("ocr")
            
display(results)

StatementMeta(synapsemlpool, 25, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, a50aec14-8f9d-4fe6-adce-e7f9f5cc161d)

***  Image Analysis ***

The image analysis API detects objects within an image like faces, landmarks, etc. and can be used to generate descriptiosn of images.

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/overview-image-analysis

In [4]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test1.jpg", "en"),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test2.png", None),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test3.png", "en")
    ], ["image", "language"])


ai = (AnalyzeImage()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setImageUrlCol("image")
        .setLanguageCol("language")
        .setVisualFeatures(["Categories", "Tags", "Description", "Faces", "ImageType", "Color", "Adult", "Objects", "Brands"])
        .setDetails(["Celebrities", "Landmarks"])
        .setOutputCol("features"))

results = ai.transform(df).select("features")
display(results)

StatementMeta(synapsemlpool, 25, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 00357214-e62a-49f3-8361-d217d028fd47)

In [5]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test1.jpg", ),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test2.png", ),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test3.png", )
    ], ["url", ])

rt = (RecognizeText()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setImageUrlCol("url")
        .setMode("Printed")
        .setOutputCol("ocr")
        .setConcurrency(5))

results = rt.transform(df).select("ocr")
display(results)

StatementMeta(synapsemlpool, 25, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, a44e5c0a-df44-4cc7-89f6-4f52d64ad572)

Note...  this is the current version of the API and is recommended for use over the OCR and RecognizeText variants.

In [6]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test1.jpg", ),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test2.png", ),
        ("https://mmlspark.blob.core.windows.net/datasets/OCR/test3.png", )
    ], ["url", ])

ri = (ReadImage()
    .setSubscriptionKey(cogServicesKey)
    .setLocation(serviceLocation)
    .setImageUrlCol("url")
    .setOutputCol("ocr")
    .setConcurrency(5))

results = ri.transform(df).select("ocr")
display(results)

StatementMeta(synapsemlpool, 25, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2429e3b3-fb1d-48d8-8bad-6396f75fb22c)

In [7]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/DSIR/test2.jpg", )
    ], ["url", ])

celeb = (RecognizeDomainSpecificContent()
        .setSubscriptionKey(cogServicesKey)
        .setModel("celebrities")
        .setLocation(serviceLocation)
        .setImageUrlCol("url")
        .setOutputCol("celebs"))

results = celeb.transform(df).select("celebs")
display(results)

StatementMeta(synapsemlpool, 25, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, b9ccb793-ce6c-48f8-8b0a-b965173639da)

In [8]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/DSIR/test1.jpg", )
    ], ["url", ])

gt = (GenerateThumbnails()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setHeight(50)
        .setWidth(50)
        .setSmartCropping(True)
        .setImageUrlCol("url")
        .setOutputCol("thumbnails"))

results = gt.transform(df)
display(results)

StatementMeta(synapsemlpool, 25, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 037aeb5d-2a71-4dea-945b-80f6fe6db958)

In [11]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/DSIR/test1.jpg", )
    ], ["url", ])

ti = (TagImage()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setImageUrlCol("url")
        .setOutputCol("tags"))

results = ti.transform(df).select("tags")
display(results)

StatementMeta(synapsemlpool, 25, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, a0c629c8-0d9a-445e-87d4-c3dfad6283c9)

In [10]:
df = spark.createDataFrame([
        ("https://mmlspark.blob.core.windows.net/datasets/DSIR/test1.jpg", )
    ], ["url", ])

di = (DescribeImage()
        .setSubscriptionKey(cogServicesKey)
        .setLocation(serviceLocation)
        .setMaxCandidates(3)
        .setImageUrlCol("url")
        .setOutputCol("descriptions"))

results = di.transform(df)
display(results)

StatementMeta(synapsemlpool, 25, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, fc37c9a8-fcf5-423e-bca8-0c021712037b)